###### Introduction to Network Analysis 2023/24 (ii)

## Network representations, basic network algorithms

You are given four networks in Pajek format that was presented in lectures.

+ Tiny toy network for testing ([toy.net](http://lovro.fri.uni-lj.si/ina/nets/toy.net))
+ Zachary karate club network ([karate_club.net](http://lovro.fri.uni-lj.si/ina/nets/karate_club.net))
+ IMDb actors collaboration network ([collaboration_imdb.net](http://lovro.fri.uni-lj.si/ina/nets/collaboration_imdb.net))
+ A small part of Google web graph ([www_google.net](http://lovro.fri.uni-lj.si/ina/nets/www_google.net))

### II. Basic network statistics

1. **(code)** Compute basic statistics of all four networks. Namely, the number of nodes $n$ and links $m$, the average node degree $\langle k\rangle=2m/n$ and the undirected density $\rho=m/{n\choose 2}$. Are the results expected?

In a graph theory context, density is a measure that shows how close a graph is to being a complete graph. For an undirected graph, the density is defined as the ratio of the number of edges m to the number of possible edges. For a simple undirected graph with n vertices, the number of possible edges is ${n\choose 2} = \frac{n*(n-1)}{2}$, which makes the formula for density become:
$\rho=m/{n\choose 2}$

However, for large values of n, the binomial coefficient ${n\choose 2}=\frac{n*(n-1)}{2}$ can cause overflow issues.

To avoid this, we can rewrite the formula to avoid calculating the binomial coefficient directly:
$\rho=m/{n\choose 2} = \frac{m}{n*(n-1)/2} = 2m/n/(n-1)$

#### 1. Solution with its own implementation

In [1]:
# Iterate over a list of graph names
for name in ["toy", "karate_club", "collaboration_imdb", "www_google"]:
    # Initialize the graph (G), number of nodes (n), and number of edges (m) for each graph
    G, n, m = None, 0, 0

    # Open the corresponding '.net' file for each graph
    with open("./networks/" + name + ".net", 'r') as file:
        # Extract the number of nodes from the first line of the file and initialize the graph (G) as a list of empty lists
        n = int(file.readline().split()[1])
        G = [[] for _ in range(n)]

        # Skip the part of the file that contains node information, stopping when reaching the line starting with "*"
        for line in file:
            if line.startswith("*"):
                break

        # Initialize edge counter (m) to zero
        m = 0
        # Process the rest of the file, which contains edges information
        for line in file:
            # Convert the first two fields in each line to integers (adjusting for 1-indexing), representing a pair of nodes with an edge between them
            i, j = (int(x) - 1 for x in line.split()[:2])
            # Add the nodes to each other's adjacency list in the graph, and increment the edge counter
            G[i].append(j)
            G[j].append(i)
            m += 1

    # Print statistics about the graph: name, number of nodes, number of edges, average degree, and density
    print("{:>10s} | '{:s}'".format('Graph', name))
    print("{:>10s} | {:,d}".format('Nodes', n))
    print("{:>10s} | {:,d}".format('Edges', m))
    print("{:>10s} | {:.2f}".format('Degree', 2 * m / n))
    print("{:>10s} | {:.2e}".format('Density', 2 * m / n / (n - 1)))
    print()

     Graph | 'toy'
     Nodes | 5
     Edges | 4
    Degree | 1.60
   Density | 4.00e-01

     Graph | 'karate_club'
     Nodes | 34
     Edges | 78
    Degree | 4.59
   Density | 1.39e-01

     Graph | 'collaboration_imdb'
     Nodes | 17,577
     Edges | 287,074
    Degree | 32.66
   Density | 1.86e-03

     Graph | 'www_google'
     Nodes | 875,713
     Edges | 5,105,039
    Degree | 11.66
   Density | 1.33e-05



#### 1. Solution using NetworkX

In [2]:
import networkx as nx

def compute_network_statistics(file_url):
    # The function nx.read_pajek reads the Pajek format file and creates a NetworkX
    # graph from it. The graph is assigned to G.
    G = nx.read_pajek(file_url)

    # Get the number of nodes in the graph.
    n = G.number_of_nodes()
    # Get the number of edges in the graph.
    m = G.number_of_edges()
    avg_k = 2 * m / n
    density = 2 * m / n / (n - 1)

    return {
        'Number of Nodes': n,
        'Number of Edges': m,
        'Average Degree': avg_k,
        'Density': density
    }

for name in ["toy", "karate_club", "collaboration_imdb", "www_google"]:
    stats = compute_network_statistics("./networks/" + name + ".net")
    print(f"For network {name}: {stats}")

For network toy: {'Number of Nodes': 5, 'Number of Edges': 4, 'Average Degree': 1.6, 'Density': 0.4}
For network karate_club: {'Number of Nodes': 34, 'Number of Edges': 78, 'Average Degree': 4.588235294117647, 'Density': 0.13903743315508021}
For network collaboration_imdb: {'Number of Nodes': 17577, 'Number of Edges': 287074, 'Average Degree': 32.6647323206463, 'Density': 0.0018584849977609412}
For network www_google: {'Number of Nodes': 875713, 'Number of Edges': 5105039, 'Average Degree': 11.659160021605253, 'Density': 1.3313920583028727e-05}


| Graph              | Nodes    | Edges      | Degree | Density   |
|--------------------|----------|------------|--------|-----------|
| toy                | 5        | 4          | 1.60   | 4.00e-01  |
| karate_club        | 34       | 38         | 4.59   | 1.39e-01  |
| collaboration_imdb | 17,577   | 287,074    | 32.66  | 1.86e-03  |
| www_google         | 875,713  | 5,105,039  | 11.66  | 1.33e-05  |

<br>

**Toy network**: This is a very small network. The average degree and density seem consistent for such a small network.


**Zachary Karate Club**: This is a well-known social network of a karate club, where each node corresponds to a member of the club, and each edge represents a tie between two members of the club. The average degree is around 4.59, which makes sense considering that it's a social network.


**IMDb actors collaboration network**: This is a much larger network, as it represents collaborations between actors on IMDb. The average degree is 32.66, suggesting that each actor has collaborated with an average of 32 other actors. Given the collaborative nature of the film industry, this seems plausible.


**Google web graph**: This represents a portion of the Google web graph, with nodes representing webpages and edges representing hyperlinks. The average degree is 11.66, suggesting that each webpage links to an average of around 11 other pages. Considering the vast and interconnected nature of the web, this also seems reasonable. The density is extremely low, reflecting the sparse connectivity of the World Wide Web.

2. **(code)** Compute the number of isolated nodes and the number of pendant nodes (i.e. degree-$1$ nodes), and the maximum node degree $k_{\rm max}$. How do the values of $k_{\rm max}$ compare to $\langle k\rangle$?

#### 1. Solution with its own implementation

In [3]:
# This function checks if a node is isolated in a graph.
# A node is considered isolated if it has no neighbors, i.e., it's not connected to any other nodes.
def isolated(G, i):
    for j in G[i]:
        if j != i:
            return False
    return True

for name in ["toy", "karate_club", "collaboration_imdb", "www_google"]:
    G, n, m = None, 0, 0

    with open("./networks/" + name + ".net", 'r') as file:
        n = int(file.readline().split()[1])
        G = [[] for _ in range(n)]

        for line in file:
            if line.startswith("*"):
                break

        m = 0
        for line in file:
            i, j = (int(x) - 1 for x in line.split()[:2])
            G[i].append(j)
            G[j].append(i)
            m += 1

    isolated_nodes, pendant_nodes, max_node_degree = 0, 0, 0
    # Calculate the degree of each node (i.e., the number of its neighbors).
    for i in range(n):
        if isolated(G, i):
            isolated_nodes += 1
        elif len(G[i]) == 1:
            pendant_nodes += 1
        if len(G[i]) > max_node_degree:
            max_node_degree = len(G[i])

    print("{:>20s} | '{:s}'".format('Graph', name))
    print("{:>20s} | {:,d}".format('Nodes', n))
    print("{:>20s} | {:,d}".format('Isolated nodes', isolated_nodes))
    print("{:>20s} | {:,d}".format('Pendant nodes', pendant_nodes))
    print("{:>20s} | {:,d}".format('Edges', m))
    print("{:>20s} | {:.2f}".format('Degree', 2 * m / n))
    print("{:>20s} | {:.2f}".format('Max node degree', max_node_degree))
    print("{:>20s} | {:.2e}".format('Density', 2 * m / n / (n - 1)))
    print()

               Graph | 'toy'
               Nodes | 5
      Isolated nodes | 1
       Pendant nodes | 1
               Edges | 4
              Degree | 1.60
     Max node degree | 3.00
             Density | 4.00e-01

               Graph | 'karate_club'
               Nodes | 34
      Isolated nodes | 0
       Pendant nodes | 1
               Edges | 78
              Degree | 4.59
     Max node degree | 17.00
             Density | 1.39e-01

               Graph | 'collaboration_imdb'
               Nodes | 17,577
      Isolated nodes | 0
       Pendant nodes | 475
               Edges | 287,074
              Degree | 32.66
     Max node degree | 784.00
             Density | 1.86e-03

               Graph | 'www_google'
               Nodes | 875,713
      Isolated nodes | 0
       Pendant nodes | 130,912
               Edges | 5,105,039
              Degree | 11.66
     Max node degree | 6353.00
             Density | 1.33e-05



#### 1. Solution using NetworkX

In [4]:
def compute_additional_statistics(file_url):
    G = nx.read_pajek(file_url)

    degrees = dict(G.degree())
    max_k = max(degrees.values())
    isolated_nodes = len(list(nx.isolates(G)))
    pendant_nodes = list(degrees.values()).count(1)  # nodes with degree 1

    return {
        'Number of Isolated Nodes': isolated_nodes,
        'Number of Pendant Nodes': pendant_nodes,
        'Maximum Degree': max_k
    }

for name in ["toy", "karate_club", "collaboration_imdb", "www_google"]:
    stats = compute_additional_statistics("./networks/" + name + ".net")
    print(f"For network {name}: {stats}")


For network toy: {'Number of Isolated Nodes': 1, 'Number of Pendant Nodes': 1, 'Maximum Degree': 3}
For network karate_club: {'Number of Isolated Nodes': 0, 'Number of Pendant Nodes': 1, 'Maximum Degree': 17}
For network collaboration_imdb: {'Number of Isolated Nodes': 0, 'Number of Pendant Nodes': 475, 'Maximum Degree': 784}
For network www_google: {'Number of Isolated Nodes': 0, 'Number of Pendant Nodes': 130912, 'Maximum Degree': 6353}


| Graph              | Isolated nodes | Pendant Nodes | Maximum degree |
|--------------------|----------------|---------------|----------------|
| toy                | 1              | 1             | 3              |
| karate_club        | 0              | 1             | 17             |
| collaboration_imdb | 0              | 475           | 784            |
| www_google         | 0              | 130,912       | 6,353          |

<br>

By comparing $k_{\rm max}$ to $\langle k\rangle$, we can get a sense of the heterogeneity of the graph's connectivity. If $k_{\rm max}$ is much larger than $\langle k\rangle$, this indicates that the graph has a few nodes that are much more interconnected than average, which can be characteristic of scale-free networks.

The number of isolated nodes is an indicator of how many nodes have no connections at all. Interestingly, only the 'toy' graph has an isolated node. Similarly, pendant nodes are those connected to only one other node.

Lastly, the density is a measure of the proportion of possible connections that are actually present in the graph. This value is typically very small for large graphs, because the number of possible connections grows quadratically with the number of nodes, but most real-world graphs are sparse and have far fewer connections.

**NOTE**: Isolated node can either have no links or one link, which connects into itself!

3. **(discuss)** What is the time complexity of the computations above?

Time complexity of the above computations is either **constant** (O(1)) or **linear** (O(n)).